<a href="https://colab.research.google.com/github/angirov/pubmed_crawler/blob/main/similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [9]:
file_name = '/gdrive/MyDrive/dsr/pubmed_data/Node2Vec_3.csv'

In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name, sep=',', index_col=0)
authors = list(df.T.index)

In [11]:
df

,Doudna JA,Savage DF,Armstrong M,Harris AR,Hsu PD,Switz NA,Nogales E,Fletcher DA,Ott M,Kumar GR,...,Savar NS,Shengjuler D,Fletcher SJ,Chang JCJ,Broadhurst DJ,Klippsten S,Sharp PP,Wenzell NA,Kuzuoglu-Ozturk D,Wang HY
0,0.315192,0.354724,0.009569,0.235932,0.615801,-0.026697,0.731761,0.553151,0.530807,-0.052219,...,0.487495,0.588571,0.781149,0.885299,0.831242,0.808671,0.739073,0.738416,0.802523,1.141953
1,-0.240204,-0.579040,-0.513756,-0.273716,-0.856893,-0.628693,-0.837647,-0.659372,-0.535281,-0.831618,...,-0.126539,-0.311865,-0.358015,-0.950293,-1.123105,-1.162375,-1.283026,-1.406187,-1.497604,-1.448709
2,0.489208,0.599460,0.602656,0.832357,-0.100672,0.367135,0.765580,0.814351,0.228257,0.101946,...,0.121302,0.010956,0.162834,0.161410,0.261680,0.217361,0.182716,0.029475,-0.217626,-0.371111
3,0.491443,0.959173,0.702689,0.091220,0.590147,0.640613,0.668798,0.632725,0.882241,0.988442,...,1.048739,1.186748,1.106415,0.615263,0.316328,0.023177,-0.212196,-0.337690,-0.239304,-0.318950
4,-0.553535,-0.637466,-0.129776,0.117218,0.032649,-0.657591,-0.546727,-0.409147,-0.326836,-0.496693,...,0.790270,0.370862,0.334017,-0.063593,-0.289258,-0.269837,-0.285863,-0.267084,-0.296049,-0.157360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.207927,-0.143652,-0.556377,-0.513526,-0.370521,-0.365829,0.106206,0.269634,0.179508,-0.692643,...,-0.144942,-0.868037,-0.969515,-0.684725,-0.585518,-0.369234,-0.107412,-0.026374,0.203261,0.601199
60,-0.412734,-0.530073,-0.978618,-0.190301,-0.204803,-0.544840,-0.696192,-0.703643,-0.190798,-0.128795,...,-0.770081,-0.730135,-0.649520,-0.750120,-0.810805,-0.898143,-0.986592,-1.082293,-0.990046,-0.820683
61,-0.121234,0.115718,-0.603727,-0.600822,0.375025,-0.390478,-0.661121,-0.018235,-0.062948,-0.027306,...,-0.287816,-0.422865,-0.482023,-0.209730,-0.311338,-0.269750,-0.329131,-0.274826,-0.337427,-0.057011
62,-0.361251,-0.358387,0.592882,0.084314,-0.079703,0.349736,0.746731,0.287138,-0.302226,-0.483793,...,-0.768946,-0.803601,-0.660016,0.091788,0.208449,0.084913,-0.066726,-0.322618,-0.521713,-0.833449


In [ ]:
authors

In [13]:
array = np.loadtxt(file_name, skiprows=1, delimiter=',')
array = np.delete(array, 0, axis = 1).T
print(array.shape)

(9309, 64)


In [24]:
import pickle

with open('authors_list.pkl','wb') as f:
    pickle.dump(authors,f)

In [ ]:
with open('authors_list.pkl','rb') as f:
    authors2 = pickle.load(f)

authors2

https://stackoverflow.com/questions/59992557/how-to-add-numpy-array-elements-row-wise-to-a-pandas-dataframe/

In [14]:
authors_df = pd.DataFrame({"athor": authors, "embedding": array.tolist()})

In [15]:
authors_df['embedding'] = authors_df['embedding'].apply(np.asarray)

In [16]:
type(authors_df['embedding'][0])

numpy.ndarray

https://stackoverflow.com/questions/17627219/whats-the-fastest-way-in-python-to-calculate-cosine-similarity-given-sparse-mat

In [17]:
from scipy.spatial.distance import cdist

similarities = 1 - cdist(array, array, metric='cosine')

In [18]:
sim_df = pd.DataFrame(similarities)

In [35]:
sim_df.to_pickle("/gdrive/MyDrive/dsr/pubmed_data/sim_df.pkl")

In [45]:
for i, author in authors:
  order_idx = sim_df[i].sort_values(ascending=False).index.tolist()
  ordered_authors = [authors[i] for i in order_idx]


In [49]:
ordered_authors[1:]

['Giraldez AJ',
 'Hanna R',
 'Kan MJ',
 'Collins FS',
 'Frank J',
 'Rath VL',
 'Liu JG',
 'Minhas FUAA',
 'IGI SARS-CoV-2 Consortium',
 'Chang MC',
 "Ferré-d'Amaré AR",
 'Bryant Z',
 'Yeo GW',
 'Battle DJ',
 'Schaffer DV',
 'Aggarwal AK',
 'Shih IH',
 'Arribere JA',
 'Been MD',
 'Fukunaga R',
 'Kiessling LL',
 'Rha B',
 'Karbstein K',
 'Jonas S',
 'Bikard D',
 'Ragunathan K',
 'Costa BM',
 'Grosshans C',
 'Chan JM',
 'Wohlschlegel J',
 'Batista VS',
 'Shao W',
 'Knitt DS',
 'Leary J',
 'Lehours AC',
 'Schoeniger JS',
 'Myong S',
 'Herschlag D',
 'Fenner M',
 'Greene EC',
 'Butler TK',
 'Schneidman-Duhovny D',
 'Markmiller SJ',
 'Evans JH',
 'Whitehead EH',
 'Yildiz A',
 'Aris KDP',
 'Brohawn SG',
 'Adams P',
 'Palefsky JM',
 'Ban N',
 'Masquida B',
 'Lin PC',
 'Gleason JM',
 'Ye J',
 'Chang SM',
 'Khokha MK',
 'McCammon JA',
 'Jankowsky E',
 'Gerber AS',
 'Arias-Palomo E',
 'Zilm KW',
 'Ekman FK',
 'Banfield J',
 'Vucelja M',
 'Dickinson ME',
 'Davies CW',
 'Mehle A',
 'Taubenberger JK

In [ ]:
sim_ordered_df = sim_df.transform(np.sort)

https://stackoverflow.com/questions/20444087/right-way-to-reverse-a-pandas-dataframe

In [ ]:
sim_ordered_df

,Doudna JA,Savage DF,Armstrong M,Harris AR,Hsu PD,Switz NA,Nogales E,Fletcher DA,Ott M,Kumar GR,...,Savar NS,Shengjuler D,Fletcher SJ,Chang JCJ,Broadhurst DJ,Klippsten S,Sharp PP,Wenzell NA,Kuzuoglu-Ozturk D,Wang HY
0,-0.017018,-0.041045,-0.011550,-0.023040,-0.012599,-0.036086,-0.066213,0.007127,-0.037419,0.066995,...,0.042279,0.012100,-0.004038,-0.018501,0.002569,-0.008611,-0.003697,0.014614,0.048105,0.067379
1,-0.014893,-0.030856,-0.009816,-0.006088,0.014364,-0.030496,-0.051840,0.041596,-0.018499,0.069754,...,0.058487,0.026775,0.002431,-0.002193,0.008117,0.033555,0.026636,0.034488,0.055266,0.076476
2,-0.006860,-0.021066,-0.005639,0.031654,0.015717,-0.030299,-0.040543,0.046985,-0.009603,0.073396,...,0.073078,0.044874,0.024070,0.003038,0.013336,0.038131,0.028783,0.038325,0.074681,0.086687
3,-0.004704,-0.020125,-0.005609,0.033306,0.021756,-0.021216,-0.039400,0.053182,-0.007379,0.074956,...,0.077970,0.053198,0.030694,0.044612,0.033185,0.038228,0.048928,0.061722,0.081061,0.092465
4,0.003304,-0.018631,-0.004404,0.036367,0.026327,-0.021019,-0.038837,0.058815,-0.006360,0.090342,...,0.083736,0.064514,0.031656,0.046862,0.057260,0.040506,0.058961,0.084074,0.083547,0.093321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9304,0.982216,0.839034,0.796284,0.964941,0.927392,0.847399,0.872078,0.981855,0.911787,0.979580,...,0.909419,0.909105,0.875079,0.930342,0.928337,0.929962,0.922598,0.920115,0.867658,0.915093
9305,0.982831,0.839432,0.822882,0.965157,0.966805,0.899897,0.879985,0.981946,0.976769,0.980268,...,0.910910,0.932074,0.927966,0.938148,0.931918,0.938148,0.931918,0.929962,0.922598,0.920115
9306,0.983416,0.865304,0.887364,0.967507,0.969241,0.913695,0.912963,0.983478,0.981826,0.981004,...,0.943284,0.947825,0.974588,0.978433,0.980345,0.980276,0.978050,0.977355,0.970596,0.941766
9307,0.984023,0.880100,0.923536,0.968280,0.980079,0.926197,0.916753,0.984595,0.982405,0.982580,...,0.948851,0.974588,0.981131,0.980345,0.982331,0.982331,0.980276,0.978050,0.977355,0.970596


In [ ]:
sim_reset_df = sim_ordered_df.reindex(index=sim_ordered_df.index[::-1]).reset_index(drop=True)

In [ ]:
sim_reset_df

,Doudna JA,Savage DF,Armstrong M,Harris AR,Hsu PD,Switz NA,Nogales E,Fletcher DA,Ott M,Kumar GR,...,Savar NS,Shengjuler D,Fletcher SJ,Chang JCJ,Broadhurst DJ,Klippsten S,Sharp PP,Wenzell NA,Kuzuoglu-Ozturk D,Wang HY
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.984023,0.880100,0.923536,0.968280,0.980079,0.926197,0.916753,0.984595,0.982405,0.982580,...,0.948851,0.974588,0.981131,0.980345,0.982331,0.982331,0.980276,0.978050,0.977355,0.970596
2,0.983416,0.865304,0.887364,0.967507,0.969241,0.913695,0.912963,0.983478,0.981826,0.981004,...,0.943284,0.947825,0.974588,0.978433,0.980345,0.980276,0.978050,0.977355,0.970596,0.941766
3,0.982831,0.839432,0.822882,0.965157,0.966805,0.899897,0.879985,0.981946,0.976769,0.980268,...,0.910910,0.932074,0.927966,0.938148,0.931918,0.938148,0.931918,0.929962,0.922598,0.920115
4,0.982216,0.839034,0.796284,0.964941,0.927392,0.847399,0.872078,0.981855,0.911787,0.979580,...,0.909419,0.909105,0.875079,0.930342,0.928337,0.929962,0.922598,0.920115,0.867658,0.915093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9304,0.003304,-0.018631,-0.004404,0.036367,0.026327,-0.021019,-0.038837,0.058815,-0.006360,0.090342,...,0.083736,0.064514,0.031656,0.046862,0.057260,0.040506,0.058961,0.084074,0.083547,0.093321
9305,-0.004704,-0.020125,-0.005609,0.033306,0.021756,-0.021216,-0.039400,0.053182,-0.007379,0.074956,...,0.077970,0.053198,0.030694,0.044612,0.033185,0.038228,0.048928,0.061722,0.081061,0.092465
9306,-0.006860,-0.021066,-0.005639,0.031654,0.015717,-0.030299,-0.040543,0.046985,-0.009603,0.073396,...,0.073078,0.044874,0.024070,0.003038,0.013336,0.038131,0.028783,0.038325,0.074681,0.086687
9307,-0.014893,-0.030856,-0.009816,-0.006088,0.014364,-0.030496,-0.051840,0.041596,-0.018499,0.069754,...,0.058487,0.026775,0.002431,-0.002193,0.008117,0.033555,0.026636,0.034488,0.055266,0.076476


In [5]:
import pandas as pd

df = pd.read_pickle("/gdrive/MyDrive/dsr/pubmed_data/similarity_matrix.pkl")

In [8]:
df.shape

(51, 51)